## Uvrscanje pesmi na playlist
Cilj: uvrstiti (predlagati) n - novih pesmi, ki se pojavijo na platformi, playlistam, ki imajo najvec sosednih pesmi.

1. Nakljucno izberemo n pesmi iz playlistov, te pesmi bomo napovedovali
2. Za vsako pesem izracunamo 100 najbliznjih sosedov vseh playlist
3. Izracunamo delez, na katerih playlistih se pojavi najvec sosed 

In [1]:
# Visualization

# scatter plot data + selected data
# radviz , scatterplot
# knn - podobnost 2 playlistov (predlagamo pesmi drugih playlistov - podobnih)

In [2]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd

In [3]:
def playlist(df_pl):
    # df_pl.drop_duplicates(subset=['id'], inplace=True)
    # df_pl.set_index("id", inplace=True)

    df_audio = df_pl.loc[:, ['danceability', 'energy', 'loudness', 
                      'speechiness', 'acousticness', 'instrumentalness',
                       'liveness', 'valence', 'tempo', 'duration_ms']]
    
    return df_audio, df_pl

In [4]:
def playlists(path):
    pls = []
    df = pd.read_csv(path, index_col=0)
    i = 0
    df_pl = pd.DataFrame(df.loc[df['playlist'] == i])
    while df_pl.shape[0] > 0:
        pls.append(df_pl)
        i += 1
        df_pl = pd.DataFrame(df.loc[df['playlist'] == i])
    
    tab_audio = []
    tab_df = []
    
    for pl in pls:
        df_audio, df_pl = playlist(pl)
        tab_audio.append(df_audio)
        tab_df.append(df_pl)
         
    return [tab_df, tab_audio]

In [5]:
tab_df_pl = playlists('Data\Playlist\Playlists5.csv')
tab_df = tab_df_pl[0]
audio_df = tab_df_pl[1]

In [6]:
 # PLAYLISTS
    # 0,1,2,3 - classical
    # 4,5,6,7 - rap
    # 8,9 - edm
    # 10 - techno
    # 11 - House
    # 12,13,14,15 - rock
    # 16,17,18,19 - latin
    # 20,21,22,23 - pop

Iz 1. playlista nakljucno izberemo 1 pesem, ki jo bomo napovedovali. 

S KNN algoritmom bomo izracunalni njenih 200 najbliznjih sosed (izmed vseh pesmi na vseh playlistih, ki jih imamo)

In [41]:
song = audio_df[6].sample()

In [42]:
df_all = pd.concat(audio_df)
df = pd.concat(tab_df)

Izmed vseh pesmi z vseh playlistov, ki jih imamo izracunamo 200 najbliznjih sosed pesmi.

In [43]:
nbrs = NearestNeighbors(n_neighbors=200).fit(df_all)
distances, indices = nbrs.kneighbors(song)
print(indices)
# print(distances)

[[ 535  435  577  570 1324  391  547  562  455  542  546  437  593  406
   436  559  538  441  491  498  578  517  423  478  413  453 1942  533
   511  400  584  505  516  599 1361  518  466  532  541  485  426 1538
   571  445  417  456  495  442 1283  412  601  548  558 1383  561  501
   448  409  377  422 1303 1277  465  596  393  410  521  420  438 1473
   497 1372  537  565  496 1628  404  569  403 1561 1344  390  524  526
   492  418  432  407  591 1804 1317  572  384  388  769  482  557 1863
   468 1299 1687  451  480  380  583  594  500 1774 1386 1540  536 1289
  1316  514  563  392  375 1708  567  381  600  376  471  588 1522 1459
   444  551  402  580  460  396  433  374  452  506  564  576 1900 1315
   477  520  618  789 1770  414  464  566 1852  378  560  411  573  467
   587 1412 1472 2003  459  486 1347 1332  475 1346 1987  586 1996  582
  1290 1716 1273  504 1463 1284 1727 1794  574 1513 1854 1626 1641 1297
  1850 1861 1352  581  472 1624 1274  528  469  449 1517 1630  4

In [44]:
print("Pesem: ", df.loc[song.index[0]]['name'])
print("10 najbolj podobnih pesmi: \n")
for x1 in indices[0][1:10]:
    print("PL: ", df.iloc[x1]['playlist'], "Song name: ", df.iloc[x1]['name'])

Pesem:  2MS
10 najbolj podobnih pesmi: 

PL:  5 Song name:  Life Is Good (feat. Drake)
PL:  7 Song name:  Hot Shower
PL:  6 Song name:  Airmax's
PL:  16 Song name:  Demasiadas Mujeres
PL:  4 Song name:  Ice Cream (feat. Rick Ross)
PL:  6 Song name:  Black Beatles
PL:  6 Song name:  I Wanna Start My Life Again
PL:  5 Song name:  Ball If I Want To
PL:  6 Song name:  22 Carats


Za pesem zanra klasika smo poisklai 10 najbolj podobnosih sosedov. Sosedi so iz 0 in 2 playlista, ki sta tudi klasika. 

Pomeni, da napovedi za 10 najbolj podobnih pesmi delujejo vredu.

In [45]:
pl_cnt  = [(0, 0)] * len(tab_df)
for row, dist in zip(indices[0], distances[0]):
    id1 = df_all.index[row]
    pl = df[(df.index == id1)]['playlist'][0]
    pl_cnt[pl] = (pl_cnt[pl][0] + 1, pl)

for index, x in enumerate(pl_cnt):
    pl_cnt[index] = (x[0] / tab_df[index].shape[0], index)

In [46]:
for y in sorted(pl_cnt, reverse=True)[0:3]:
    print("Playlist: %d Delez najbliznjih sosedov: %.f%%" % (y[1], y[0]*100))


Playlist: 7 Delez najbliznjih sosedov: 67%
Playlist: 6 Delez najbliznjih sosedov: 64%
Playlist: 4 Delez najbliznjih sosedov: 58%


Pesem bi predlagali playlistu 2,1,0 ker ima pesem na teh playlistih najvecji delez podobnih sosedov.

Za pesem zanra klasika bi uvrstili na playliste 0,1,2 (playlisti klasike), pomeni, da bi pesem napovedali pravemu uporabniku.